In [32]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [33]:
import pandas as pd
import numpy as np

In [34]:
# only keep the useful data and save it 
sa_pos_2016 = spark.read.csv('../data/tables/CG_POA_2016_SA2_2016.csv')
sa_pos_2016.limit(5)
from pyspark.sql import functions as F
sa_pos_2016 = sa_pos_2016.filter(
    (F.col('_c1') != 'null' )
    & (F.col('_c5') != 'null')
)
cols = ['_c6','_c7','_c8','_c9','_c10','_c11','_c12']
sa_pos_2016 = sa_pos_2016.drop(*cols)
sa_pos_2016.write.mode('overwrite').csv('../data/tables/sa_pos_2016.csv')

In [35]:
# read the saved data
sa_pos_2016 = spark.read.csv('../data/tables/sa_pos_2016.csv',header=True)
sa_pos_2016 = sa_pos_2016.select('POA_CODE_2016','SA2_MAINCODE_2016')

In [40]:
# read poa and sa2 of 2016 & 2021
poa_2016_2021 = spark.read.csv('../data/tables/poa_2016_2021.csv',header=True)
sa2_2016_2021 = spark.read.csv('../data/tables/sa2_2016_2021.csv',header=True)
poa_2016_2021 = poa_2016_2021.select('POA_CODE_2016','POA_CODE_2021')
sa2_2016_2021 = sa2_2016_2021.select('SA2_MAINCODE_2016','SA2_CODE_2021')
poa_2016_2021.limit(3)
print(poa_2016_2021.count())


2887


In [41]:
sa2_2016_2021

SA2_MAINCODE_2016,SA2_CODE_2021
101021007,101021007
101021008,101021008
101021009,101021009
101021010,101021010
101021011,101021610
101021011,101021611
101021012,101021012
101031013,101031013
101031014,101031014
101031015,101031015


In [42]:
poa_2016_2021

POA_CODE_2016,POA_CODE_2021
0800,0800
0810,0810
0812,0812
0815,0810
0820,0820
0822,0837
0822,0822
0828,0828
0829,0829
0830,0830


In [43]:
# merge them by postcode
table = sa_pos_2016.join(poa_2016_2021,sa_pos_2016.POA_CODE_2016 == poa_2016_2021.POA_CODE_2016,'inner').drop(poa_2016_2021.POA_CODE_2016)
table.limit(5)

POA_CODE_2016,SA2_MAINCODE_2016,POA_CODE_2021
0800,701011002,0800
0810,701021010,0810
0810,701021013,0810
0810,701021016,0810
0810,701021018,0810


In [44]:
#  merge them by sa2 code
table = table.join(sa2_2016_2021,table.SA2_MAINCODE_2016 == sa2_2016_2021.SA2_MAINCODE_2016,'inner').drop(sa2_2016_2021.SA2_MAINCODE_2016)
table.limit(5)

POA_CODE_2016,SA2_MAINCODE_2016,POA_CODE_2021,SA2_CODE_2021
0800,701011002,0800,701011002
0810,701021010,0810,701021010
0810,701021013,0810,701021013
0810,701021016,0810,701021016
0810,701021018,0810,701021018


In [45]:
# check number of postcode
table = table.drop('POA_CODE_2016','SA2_MAINCODE_2016')
table = table.select('POA_CODE_2021').distinct()
print(table.count(),len(table.columns))

2642 1


In [46]:
table.write.mode('overwrite').option("header",True).csv('../data/tables/sa_pos_2021.csv')